# Vanilla h=64 Seed Expansion
**Goal**: Train 20 more seeds (s45-s64) at the winning vanilla h=64 recipe.
Already have s42, s43, s44 from parity audit → total 23 seeds.

**Recipe**: vanilla+linear, h=64, 3L, raw32, no norm, MSE, dropout=0 → 0.2814 LB (PB!)

At 184s/model on scoring server, 15-20 model ensemble fits in budget.
With ONNX (~73s/model), can fit 40+ models.

In [ ]:
# Cell 0: Mount Drive, download data from Kaggle
import os, json

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "FILL_IN"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

In [ ]:
# Cell 1: Setup — clone repo, link data
import os, subprocess
REPO = "/content/competition_package"

os.chdir("/content")
subprocess.run(["rm", "-rf", REPO], check=False)
subprocess.run(["git", "clone", "https://github.com/vincentvdo6/competition_package.git", REPO], check=True)
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

subprocess.run(["ln", "-sf", "/content/data/train.parquet", "datasets/train.parquet"], check=True)
subprocess.run(["ln", "-sf", "/content/data/valid.parquet", "datasets/valid.parquet"], check=True)

assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"
commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip()
print(f"Commit: {commit}")
print(f"GPU: {subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], text=True).strip()}")
print("Ready!")

In [ ]:
# Cell 2: Train seeds s45-s64 (20 seeds)
import os, subprocess, sys
os.chdir("/content/competition_package")

CONFIG = "configs/gru_parity_v1.yaml"
SEEDS = list(range(45, 65))  # s45-s64

print(f"=== VANILLA h=64 SEED EXPANSION ===")
print(f"Config: {CONFIG}")
print(f"Seeds: {SEEDS[0]}-{SEEDS[-1]} ({len(SEEDS)} seeds)")
print("=" * 60, flush=True)

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"Training gru_parity_v1 seed {seed}")
    print(f"{'='*60}", flush=True)
    proc = subprocess.Popen(
        [sys.executable, "-u", "scripts/train.py",
         "--config", CONFIG,
         "--seed", str(seed), "--device", "cuda"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    proc.wait()
    if proc.returncode != 0:
        print(f"ERROR: seed {seed} failed with return code {proc.returncode}")

print(f"\nAll {len(SEEDS)} seeds done!")

In [ ]:
# Cell 3: Evaluation — rank all seeds
import os, glob, torch
os.chdir("/content/competition_package")

PARITY_V1_LB = 0.2814   # s43 single model LB
PARITY_V1_VAL = 0.2692   # mean val from parity audit

results = []
for pt in sorted(glob.glob("logs/gru_parity_v1_seed*.pt")):
    basename = os.path.basename(pt)
    if '_epoch' in basename:
        continue
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    score = float(ckpt.get("best_score", 0))
    epoch = ckpt.get("best_epoch", "N/A")
    seed = int(basename.split("seed")[1].split(".")[0])
    results.append((seed, basename, score, epoch))

results.sort(key=lambda x: -x[2])  # Sort by val score descending

print(f"{'Rank':<6} {'Seed':<6} {'Val Score':>10} {'Epoch':>6}")
print("-" * 35)
for i, (seed, name, score, epoch) in enumerate(results, 1):
    marker = " ***" if score >= 0.2737 else (" **" if score >= 0.2700 else "")
    print(f"{i:<6} s{seed:<5} {score:>10.4f} {str(epoch):>6}{marker}")

scores = [s for _, _, s, _ in results]
mean_val = sum(scores) / len(scores)
print(f"\nTotal: {len(results)} seeds")
print(f"Mean val: {mean_val:.4f}")
print(f"Best val: {results[0][2]:.4f} (s{results[0][0]})")
print(f"Top-5 mean: {sum(scores[:5])/5:.4f}")
print(f"Top-10 mean: {sum(scores[:10])/10:.4f}")
print(f"Top-15 mean: {sum(scores[:15])/15:.4f}")
print(f"\n*** = val >= 0.2737 (matches s43 which scored 0.2814 LB)")
print(f"**  = val >= 0.2700")

In [ ]:
# Cell 4: Strip checkpoints + zip + save to Drive
import os, torch, glob, shutil
os.chdir("/content/competition_package")
os.makedirs("logs/slim", exist_ok=True)

for pt in sorted(glob.glob("logs/gru_parity_v1_seed*.pt")):
    basename = os.path.basename(pt)
    if '_epoch' in basename:
        continue
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    slim = {
        "model_state_dict": ckpt["model_state_dict"],
        "config": ckpt.get("config", {}),
        "best_score": ckpt.get("best_score", None),
        "best_epoch": ckpt.get("best_epoch", None),
    }
    out = f"logs/slim/{basename}"
    torch.save(slim, out)
    orig = os.path.getsize(pt) / 1e6
    new = os.path.getsize(out) / 1e6
    print(f"{basename}: {orig:.1f}MB -> {new:.1f}MB")

shutil.make_archive("/content/vanilla_seeds", "zip",
                     "/content/competition_package/logs/slim")
sz = os.path.getsize("/content/vanilla_seeds.zip") / 1e6
print(f"\nvanilla_seeds.zip: {sz:.1f}MB")

shutil.copy("/content/vanilla_seeds.zip",
            "/content/drive/MyDrive/wunderfund/vanilla_seeds.zip")
print("Saved to Drive: MyDrive/wunderfund/vanilla_seeds.zip")